<a href="https://colab.research.google.com/github/mboya2020/Churn-in-Telecom-s-Dataset-Classification-Modeling/blob/main/Copy_of_Call_Volume_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CALL VOLUME PREDICTION

## 1.0 Business Understanding

### 1.1  Problem statement
As a result of an increased number of reported cases on child abuse, there is a dire need to have prompt responses. It is crucial that reported cases are acted upon promptly to ensure timely and efficient handling of reported cases.







### 1.2 Business Question
The stakeholders are Mtoto News, a digital company that leverages on technology to improve child wellbeing, and Childline Kenya, an NGO established in response to the state of child protection in Kenya and the manner in which abuse cases were being reported and handled.


### 1.3 Main objective
Building a forecast model that predicts the number of incoming calls that Childline Kenya will receive per hour per day..

### 1.4 Supplementary Objective
Ensuring Childline Kenya is adequately staffed to handle reported cases.


### 1.5 Metric of success
The algorithm will be considered a success if the predictions made by it has a:

**Precision of 90-96%

## 2.0 DATA UNDERSTANDING

### 2.1 The Dataset

The dataset is provided on [Zindi Africa](https://zindi.africa/competitions/mtoto-news-childline-kenya-call-volume-prediction-challenge), the training data contains all the calls (over 135,000) that were received from 1 January 2016 to 12 July 2016.

Each call contains the following fields:

- calldate - Date (month-day-year) and time of the call

- cc_status - Case status

- maincat - Main category call falls into

- subcat1 - Subcategory call falls under

- casepriority - Priority of the case

- referal -  Place case referred to

- caller_gender - Gender of the caller

- caller_age - Age of the caller

- caller_county - Area where the call came from

- child_age - Age of the child in case

- child_gender - Gender of the child in case

- child_county - Area where the child is from

- parent_age - Age of the parent

- parent_gender - Gender of the parent

- parent_county - Area where the child is from

- Abuser_Relationship - Relationship abuser has with the child in case

- Neglector_Relationship - Relationship neglector has with the child in case

- Physical_abuser_Relationship - Relationship physical abuser has with the child in case.

An explanation of all the columns of the weather data can be found <a href= "https://rp5.ru/Weather_archive_in_Nairobi_(airport)">here</a>

## 3.0 Data Preparation

### 3.1 Loading libraries

Connecting to Google drive and Importing all the necessary Libraries | Modules.

In [1]:
#connecting collab to google drive 

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import all the relevant libraries

import pandas as pd
import numpy as np
import os, shutil

# visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline




### 3.2 Loading the data

The data was loaded using `os.listdir` method

In [3]:
# print the contents of the mtoto news childline kenya folder 

print(os.listdir('/content/drive/Shareddrives/Eagles/mtoto_news_childline_kenya'))

['NairobiSchoolDates2016.csv', 'Sample_Submission.csv', 'KenyaPublicHolidays2016.csv', 'train.csv', 'WeatherNairobi2016.xlsx', 'train.gsheet']


In [4]:
#store the file paths as variables

weather_file = ('/content/drive/Shareddrives/Eagles/mtoto_news_childline_kenya/WeatherNairobi2016.xlsx')
dates_file = ('/content/drive/Shareddrives/Eagles/mtoto_news_childline_kenya/NairobiSchoolDates2016.csv')
holidays_file = ('/content/drive/Shareddrives/Eagles/mtoto_news_childline_kenya/KenyaPublicHolidays2016.csv')
train_file = ('/content/drive/Shareddrives/Eagles/mtoto_news_childline_kenya/train.csv')

View contents of the files

In [5]:
#print the data in the train file 

train_data = pd.read_csv(train_file)
train_data.head(3)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (12,13,14,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,calldate,cc_status,maincat,subcat1,casepriority,referal,caller_gender,caller_age,caller_county,child_age,child_gender,child_county,parent_age,parent_gender,parent_county,Abuser_Relationship,Neglector_Relationship,Perpetrator_Relationship,Physical abuser_Relationship
0,2016-01-01 07:26:47,Closed,non-interventional,Blank call,Non Critical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01 07:32:30,Closed,non-interventional,Blank call,Non Critical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-01 07:44:31,Closed,non-interventional,Blank call,Non Critical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#print the data in the dates file 

date_data = pd.read_csv(dates_file)
date_data.head(3)

,Term,Opening,Closing,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,1,2016-01-04,2016-04-08,NaN,NaN,*year/month/day
1,2,2016-05-02,2016-08-05,NaN,NaN,https://www.standardmedia.co.ke/article/200018...
2,3,2016-09-05,2016-11-18,NaN,NaN,NaN


In [7]:
#print the data in the holidays file 

holiday_data = pd.read_csv(holidays_file, skiprows= 2)
holiday_data.head()

,Date,Holiday,Unnamed: 2
0,2016/01/01,New Years Day,NaN
1,2016/03/25,Good Friday,NaN
2,2016/03/28,Easter Monday,NaN
3,2016/05/01,Labour Day,NaN
4,2016/05/02,Labour Day observed,NaN


In [8]:
# uncomment following line to install pandas update of xlrd
# !pip install --upgrade xlrd
weather_data= pd.read_excel(weather_file, skiprows= 6)

In [9]:
weather_data.head()

,Local time in Nairobi (airport),T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,Cm,Ch,VV,Td,RRR,tR,E,Tg,E',sss
0,31.12.2016 21:00,20.2,NaN,NaN,NaN,68.0,Wind blowing from the east-northeast,8.0,NaN,NaN,...,Altocumulus translucidus at a single level.,"No Cirrus, Cirrocumulus or Cirrostratus.",30.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN
1,31.12.2016 15:00,26.0,NaN,NaN,NaN,40.0,Wind blowing from the north-east,9.0,NaN,NaN,...,"No Altocumulus, Altostratus or Nimbostratus.","No Cirrus, Cirrocumulus or Cirrostratus.",30.0,11.2,NaN,NaN,NaN,NaN,NaN,NaN
2,31.12.2016 09:00,18.8,NaN,NaN,NaN,85.0,Wind blowing from the north-northeast,5.0,NaN,NaN,...,Altocumulus translucidus at a single level.,"No Cirrus, Cirrocumulus or Cirrostratus.",25.0,16.2,Trace of precipitation,24.0,NaN,NaN,NaN,NaN
3,30.12.2016 21:00,19.5,NaN,NaN,NaN,79.0,Wind blowing from the east-northeast,4.0,NaN,NaN,...,Altocumulus translucidus at a single level.,"No Cirrus, Cirrocumulus or Cirrostratus.",20.0,15.8,NaN,NaN,NaN,NaN,NaN,NaN
4,30.12.2016 15:00,24.5,NaN,NaN,NaN,56.0,Wind blowing from the east-northeast,7.0,NaN,NaN,...,Altocumulus translucidus at a single level.,Cirrus spissatus cumulonimbogenitus.,30.0,15.1,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
train_data['maincat'].unique()

array(['non-interventional', 'others', 'general_information',
       'family_community_society', 'health_basic_needs', 'sexuality',
       'abuse', 'education_career'], dtype=object)

### 3.3 Data Cleaning

#### 3.3.0 Train data Cleaning

Checking for 

1.   Validity
2.   Completenes
3.   Consistency
4.   Uniformity



##### 3.3.1 Data Validity


*   Checking for the validity of data in the train_data
*   Removing  irrelevant columns 
*   Merging columns with same information 



In [11]:
# remove the last column since it has irrelevant info
holiday_data.drop('Unnamed: 2',axis= 1, inplace= True)

In [12]:
holiday_data.head()

,Date,Holiday
0,2016/01/01,New Years Day
1,2016/03/25,Good Friday
2,2016/03/28,Easter Monday
3,2016/05/01,Labour Day
4,2016/05/02,Labour Day observed


In [13]:
# remove the last 3 columns since they have irrelevant info
cols_to_drop= ['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
date_data.drop(cols_to_drop,axis= 1, inplace= True)

##### 3.3.2 Uniformity

*   Checking different columns to ensure that they are correct.
*   Merging columns with same information
*   Deleteing columns with duplicate entries



In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135988 entries, 0 to 135987
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   calldate                      135988 non-null  object
 1   cc_status                     135988 non-null  object
 2   maincat                       135988 non-null  object
 3   subcat1                       135988 non-null  object
 4   casepriority                  135988 non-null  object
 5   referal                       1262 non-null    object
 6   caller_gender                 11548 non-null   object
 7   caller_age                    10732 non-null   object
 8   caller_county                 8667 non-null    object
 9   child_age                     3979 non-null    object
 10  child_gender                  3992 non-null    object
 11  child_county                  3749 non-null    object
 12  parent_age                    33 non-null      object
 13 

We noticed that in the referal column a lot of the values of the columns are similar both in the text and meaning. We therefore merge the similar values while also rectifying spelling errors.

In [15]:
# merge values similar to children office
similar_to_children_law= ["Childrens", "Childrens Office", "Children's court", "Children's office", "CHILDREN", "Childrens'Court", "children's office", "Children's", "Child",
                          "Childrens officer", "Childrens office", "children's court", "Children's Court", "Childrens court", "children Officer"]
train_data['referal']= train_data['referal'].map(lambda x: 'children office/court' if x in similar_to_children_law else x)

# merge values similar to chief office
similar_to_chief_office= ["Chief's", "Chief's office", "Chief's Office", "Kenya", "Kenya Police", "kenya Police", "chief's office", "Chief officer",
                          "Chief", "Police station", "DO's", "Assistant county commisioners(DOs)"]
train_data['referal']= train_data['referal'].map(lambda x: 'chief/police office' if x in similar_to_chief_office else x)

# merge values similar to OVC rescue centre
similar_to_OVC= ["Children and rescuing centre(OVC)", "Children's rescuing Center(OVC)", "children's rescuing Center(OVC)", "children's rescuing center(OVC)", "OVC"]
train_data['referal']= train_data['referal'].map(lambda x: 'OVC rescue' if x in similar_to_OVC else x)

# similar to constituency fund
similar_to_constituency= ["Constituency", "Constituency Development  Fund(CDF)"]
train_data['referal']= train_data['referal'].map(lambda x: 'constituency dev fund' if x in similar_to_constituency else x)

# similar to education office
similar_to_ed= ["Education", "Education Office"]
train_data['referal']= train_data['referal'].map(lambda x: 'education office' if x in similar_to_ed else x)

# similar to youth hotline
similar_to_youth_hotline= ["Youth", "Youth Hotline 1190", "Youth hotline 1190"]
train_data['referal']= train_data['referal'].map(lambda x: 'youth hotline 1190' if x in similar_to_youth_hotline else x)

# fix a spelling error in 'child welfare sociaty'
train_data['referal']= train_data['referal'].map(lambda x: 'child welfare society' if x == 'child welfare sociaty' else x)

In [16]:
# checking that the values have been changed
train_data['referal'].value_counts()

children office/court                                 788
chief/police office                                   318
Hospital                                               38
constituency dev fund                                  38
education office                                       25
youth hotline 1190                                     22
OVC rescue                                              8
CRADDLE                                                 4
GENDER                                                  4
FIDA                                                    3
Area                                                    2
CLAN                                                    1
child welfare society                                   1
Charitable                                              1
Amani                                                   1
World                                                   1
Save                                                    1
CARE          

We repeat the process for any other columns with the same issue.

In [17]:
# fixing a capitalisation issue in casepriority
train_data['casepriority']= train_data['casepriority'].map(lambda x: 'Non Critical' if x == 'non Critical' else x)

In [18]:
# fixing a spelling error in subcategory1
train_data['subcat1']= train_data['subcat1'].map(lambda x: 'Children rights/ issues' if x == 'Childrenï¿½s rights/ issues' else x)

In [19]:
# fixing a typo in the age columns
train_data['caller_age']= train_data['caller_age'].map(lambda x: '11-15' if x == '2016-11-15 00:00:00' else x)
train_data['caller_age']= train_data['caller_age'].map(lambda x: '6-10' if x == '2016-06-10 00:00:00' else x)


In [20]:
# combine the 10-15 and 11-15 values
train_data['child_age']= train_data['child_age'].map(lambda x: '11-15' if (x== '2016-11-15 00:00:00' or x== '2016-10-15 00:00:00' or x==' 11-15') else x)
train_data['child_age']= train_data['child_age'].map(lambda x: '6-10' if x == '2016-06-10 00:00:00' else x)

In [21]:
# removing the distinction btwn certain texts that mean the same thing in the abuser relationship column
train_data['Abuser_Relationship']= train_data['Abuser_Relationship'].map(lambda x: 'Neighbor' if (x== 'Neighbour' or x== 'Neigbour') else x)
train_data['Abuser_Relationship']= train_data['Abuser_Relationship'].map(lambda x: 'Father' if x== 'father' else x)
train_data['Abuser_Relationship']= train_data['Abuser_Relationship'].map(lambda x: 'Unknown' if (x== 'Unspecified relationship' or x== 'unknown' or x== 'Undisclosed relationship' or x== 'Concerned') else x)
train_data['Abuser_Relationship']= train_data['Abuser_Relationship'].map(lambda x: 'Parent' if x== 'Parents' else x)
train_data['Abuser_Relationship']= train_data['Abuser_Relationship'].map(lambda x: 'Uncle' if x== 'Uncles' else x)
train_data['Abuser_Relationship']= train_data['Abuser_Relationship'].map(lambda x: 'Extended' if x== 'Extended Family' else x)
train_data['Abuser_Relationship']= train_data['Abuser_Relationship'].map(lambda x: 'Teacher' if x== 'Student' else x)
train_data['Abuser_Relationship']= train_data['Abuser_Relationship'].map(lambda x: 'Romantic Partner' if (x== 'Romantic parner' or x== 'Partner') else x)

In [22]:
# removing distinctions in the neglector relationship column
train_data['Neglector_Relationship']= train_data['Neglector_Relationship'].map(lambda x: 'Parent' if x== 'Parents' else x)
train_data['Neglector_Relationship']= train_data['Neglector_Relationship'].map(lambda x: 'Father' if x== 'father' else x)
train_data['Neglector_Relationship']= train_data['Neglector_Relationship'].map(lambda x: 'GrandMother' if x== 'Grandmother' else x)
train_data['Neglector_Relationship']= train_data['Neglector_Relationship'].map(lambda x: 'Step-Mother' if x== 'Step mother' else x)

In [23]:
# removing distinctions in the perpertrator relationship column
train_data['Perpetrator_Relationship']= train_data['Perpetrator_Relationship'].map(lambda x: 'Father' if (x== 'father' or x== 'Father ') else x)
train_data['Perpetrator_Relationship']= train_data['Perpetrator_Relationship'].map(lambda x: 'Mother' if (x== 'mother' or x== 'mother ') else x)
train_data['Perpetrator_Relationship']= train_data['Perpetrator_Relationship'].map(lambda x: 'Neighbor' if (x== 'neighbor' or x== 'neighbour' or x== 'Neighbour' or x== 'Neigbour') else x)
train_data['Perpetrator_Relationship']= train_data['Perpetrator_Relationship'].map(lambda x: 'Romantic partner' if (x== 'romantic partner' or x== 'Romantic Parner' or x== 'Romantic parner') else x)
train_data['Perpetrator_Relationship']= train_data['Perpetrator_Relationship'].map(lambda x: 'Extended' if x== 'Extended family' else x)
train_data['Perpetrator_Relationship']= train_data['Perpetrator_Relationship'].map(lambda x: 'Parent' if (x== 'Parents' or x== 'Parents' or x== 'parents') else x)
train_data['Perpetrator_Relationship']= train_data['Perpetrator_Relationship'].map(lambda x: 'Unknown' if (x== 'unspecified relationship' or x== 'Assistant Chief') else x)
train_data['Perpetrator_Relationship']= train_data['Perpetrator_Relationship'].map(lambda x: 'Uncle' if x== 'uncle' else x)
train_data['Perpetrator_Relationship']= train_data['Perpetrator_Relationship'].map(lambda x: 'Guardian' if x== 'guardian' else x)
train_data['Perpetrator_Relationship']= train_data['Perpetrator_Relationship'].map(lambda x: 'Step-Mother' if x== 'step-mother' or x== 'Step mother' else x)
train_data['Perpetrator_Relationship']= train_data['Perpetrator_Relationship'].map(lambda x: 'Grandmother' if x== 'grandmother' else x)
train_data['Perpetrator_Relationship']= train_data['Perpetrator_Relationship'].map(lambda x: 'Teacher' if (x== 'teacher' or x== 'student') else x)

In [24]:
# removing distinctions in the physical abuser relationship column
train_data['Physical abuser_Relationship']= train_data['Physical abuser_Relationship'].map(lambda x: 'Father' if x== 'father' else x)
train_data['Physical abuser_Relationship']= train_data['Physical abuser_Relationship'].map(lambda x: 'Mother' if x== 'mother' else x)
train_data['Physical abuser_Relationship']= train_data['Physical abuser_Relationship'].map(lambda x: 'Friend' if x== 'peer' else x)
train_data['Physical abuser_Relationship']= train_data['Physical abuser_Relationship'].map(lambda x: 'Parent' if x== 'Parents' else x)
train_data['Physical abuser_Relationship']= train_data['Physical abuser_Relationship'].map(lambda x: 'Neighbor' if (x== 'Naighbor' or x== 'Neighbour') else x)
train_data['Physical abuser_Relationship']= train_data['Physical abuser_Relationship'].map(lambda x: 'Step-Mother' if x== 'Step-Mother ' else x)

Here we merge the Abuser_Relationship, Neglector_Relationship, Physical abuser_Relationship, Perpetrator_Relationship columns as they seem to refer to the same person (the person who did the abuse)

In [25]:
# here we check if any of the rows have any data filled into more than one of the 4 columns
zipped= zip(train_data['Abuser_Relationship'], train_data['Neglector_Relationship'],
            train_data['Physical abuser_Relationship'], train_data['Perpetrator_Relationship'])
clashing_columns= 0
for i,j,k,l in zipped:
  check_for_na= not pd.isna(i) and not pd.isna(j) and not pd.isna(k) and not pd.isna(l)
  if check_for_na:
    clashing_columns+=1

In [26]:
# here we make a column 'abuser_relationship' and add values from the other 4 columns into it
train_data['abuser_relationship']= None
zipped= zip(train_data['Abuser_Relationship'], train_data['Neglector_Relationship'],
            train_data['Physical abuser_Relationship'], train_data['Perpetrator_Relationship'])
ind= 0
for i,j,k,l in zipped:
    if not pd.isna(i):
        train_data['abuser_relationship'][ind]= i
        continue
    elif not pd.isna(j):
        train_data['abuser_relationship'][ind]= j
        continue
    elif not pd.isna(k):
        train_data['abuser_relationship'][ind]= k
        continue
    elif not pd.isna(l):
        train_data['abuser_relationship'][ind]= l
    ind+=1

Aside from deleting the Abuser_Relationship, Neglector_Relationship, Physical abuser_Relationship, Perpetrator_Relationship...
we delete the parent age column, because in the values each unique numerical value only has one entry and it would therefore not help us in modeling or EDA.
We also drop the parent county column as it has very little non-null data

In [27]:
cols_to_drop= ['parent_age', 'parent_county', 'Abuser_Relationship',
               'Neglector_Relationship', 'Physical abuser_Relationship', 'Perpetrator_Relationship']
train_data.drop(cols_to_drop, axis= 1, inplace= True)

In [28]:
# observing clean train_data
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135988 entries, 0 to 135987
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   calldate             135988 non-null  object
 1   cc_status            135988 non-null  object
 2   maincat              135988 non-null  object
 3   subcat1              135988 non-null  object
 4   casepriority         135988 non-null  object
 5   referal              1262 non-null    object
 6   caller_gender        11548 non-null   object
 7   caller_age           10732 non-null   object
 8   caller_county        8667 non-null    object
 9   child_age            3979 non-null    object
 10  child_gender         3992 non-null    object
 11  child_county         3749 non-null    object
 12  parent_gender        69 non-null      object
 13  abuser_relationship  597 non-null     object
dtypes: object(14)
memory usage: 14.5+ MB


##### 3.3.3 Consistency

In [29]:
# checking for duplicates

print(f'The data has {train_data.duplicated().sum()} duplicates')

The data has 2037 duplicates


#### 3.3.0  Weather data cleaning

We delete all columns besides:
- 'T'(temperature)
- 'U'(humidity)
- 'Ff'(wind speed)
- 'N'(cloud cover)
- 'Cl'(Types of clouds present) - (stratocumulus, stratus, cumulus, cumulonimbus)
- 'H'(height of the base of the clouds that are present
- 'Cm'(more types of clouds that are present) - (altocumulus, altostratus, nimbostratus)
- 'Ch'(more types of clouds) - (cirrus, cirrocumulus, cirrostratus)

Because these are the only useful ones to our project.<br>
We also rename the columns to better represent the information they have.

In [30]:
# specifying the columns to keep
cols_to_keep= ['Local time in Nairobi (airport)','T','U', 'Ff', 'N', 'H']

# specifying column names
weather_cols= ['dateTime', 'temperature', 'humidity', 'wind_speed', 'total_cloud_cover', 'cloud_height']

weather= weather_data[cols_to_keep]
weather.columns= weather_cols

# observing clean weather data
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1367 entries, 0 to 1366
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dateTime           1367 non-null   object 
 1   temperature        1367 non-null   float64
 2   humidity           1360 non-null   float64
 3   wind_speed         1366 non-null   float64
 4   total_cloud_cover  1367 non-null   object 
 5   cloud_height       1325 non-null   object 
dtypes: float64(3), object(3)
memory usage: 64.2+ KB


For the humidity and wind_speed columns, we fill the null values with the means of their corresponding columns.<br>
For the cloud_height column, we fill the null values with the mode because its dtype is 'object'<br>
We chose this method of filling null values because it invokes the least changes in the distributions of the various columns.

In [31]:
weather['humidity'].fillna(weather['humidity'].mean(), inplace= True)
weather['wind_speed'].fillna(weather['wind_speed'].mean(), inplace= True)
weather['cloud_height'].fillna(weather['cloud_height'].value_counts().index[0], inplace= True);

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


##### 3.3.4 Completeness

*   Handling missing values in weather dataframe



In [33]:
# checking to see if missing values have been replaced

print(f'The data has {weather.isna().sum().sum()} missing values')

The data has 0 missing values


##### 3.3.5 Consistecy

*   Checking for duplicates



In [34]:
# checking for duplicates

print(f'The data has {weather.duplicated().sum()} duplicates')

The data has 0 duplicates
